# What happens:
The chinese wall file is created and the table is refreshed by itself when you open the file

## Import

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import datetime
import win32com.client as win32
import calendar
from openpyxl import Workbook, load_workbook
from IPython.lib.display import Audio
from datetime import date

In [2]:
directory = os.path.abspath(os.getcwd()) + "/"
rd = str(input("Please enter the RD (YYYYMMDD): "))

In [3]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))


In [4]:
ws_list = ['0002', '0046', '0194', '0195', '0010', '0200', '0201', '0202', '0203']
ws_list_names = ['CONS', 'CONS_NR', 'UNCONS', 'UNCONS_NR', 'RBIAG', 'CONS_RIO', 'UNCONS_RIO', 'CONS_NPE', 'UNCONS_NPE']

df_final = pd.DataFrame()

for idx, i in enumerate(ws_list):
    connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'), {partition})".format(
                        reporting_date = rd, partition = i))
    df_temp = pd.read_sql_query("""select booking_company, securitized, securitization_reference	
    from t_cdr	
    where securitized in ('T', 'U')	
    group by partition_key, booking_company, securitized, securitization_reference	
    order by booking_company, securitization_reference	
        """, connection_RAY)
    df_temp['ws'] = ws_list_names[idx]
    df_final = df_final.append(df_temp, ignore_index=True)
    print(ws_list_names[idx] + " is done")

C:\Users\WZHARBC\AppData\Local\Temp\1\ipykernel_19380\2186603911.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df_temp, ignore_index=True)


CONS is done


C:\Users\WZHARBC\AppData\Local\Temp\1\ipykernel_19380\2186603911.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df_temp, ignore_index=True)


CONS_NR is done


C:\Users\WZHARBC\AppData\Local\Temp\1\ipykernel_19380\2186603911.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df_temp, ignore_index=True)


...

C:\Users\WZHARBC\AppData\Local\Temp\1\ipykernel_19380\2186603911.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df_temp, ignore_index=True)


UNCONS_RIO is done


C:\Users\WZHARBC\AppData\Local\Temp\1\ipykernel_19380\2186603911.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df_temp, ignore_index=True)


CONS_NPE is done
UNCONS_NPE is done


C:\Users\WZHARBC\AppData\Local\Temp\1\ipykernel_19380\2186603911.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df_temp, ignore_index=True)


In [5]:
im_folder = "Import/"
ex_folder = "Export/"
cw_check = load_workbook(im_folder + "CW_check - Template.xlsx")
cw = cw_check['CW']

cell_range = cw['A2':'D200']

#clear units
for row in cell_range:
    for cell in row:
        cell.value = None  

In [6]:
for i, row in df_final.iterrows():
    cw.cell(row=i+2, column=1).value = row.ws
    cw.cell(row=i+2, column=2).value = row.booking_company
    cw.cell(row=i+2, column=3).value = row.securitized
    cw.cell(row=i+2, column=4).value = row.securitization_reference

In [7]:
pivot = cw._pivots[0]
pivot.cache.refreshOnLoad = True

In [8]:
today = date.today()
date_today = today.strftime("%Y%m%d")
file_name = "CW_check - " + date_today + ".xlsx"
cw_check.save(ex_folder + file_name)

In [9]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'karolina.cieszynska@rbinternational.com; zoltan.peterfalvi@rbinternational.com'
mail.Cc = 'nertila.gjakova@rbinternational.com;'
mail.Subject = 'Chinese Wall Check ' + date_today
mail.Body = '''Dear Karolina,

Please find attached, the monthly Chinese Wall check.

Kr
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''

# To attach a file to the email (optional):
attachment  = directory + ex_folder + file_name
mail.Attachments.Add(attachment)

mail.Display()